In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from litellm import completion
from typing import List, Dict
import json
import re


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are an expert on python programming.  When given a description of a function, create a python program that meets that description.  When you respond, respond only with a json dictionarey that has one entry for the code and different entries for any commentary"},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)

In [ ]:
import json
import os

def construct_prompt(memory, user_input, system_rules):
    """Combines memory, user input, and system rules into a prompt."""
    prompt = f"""
    Memory: {memory}
    User Input: {user_input}
    System Rules: {system_rules}

    Please provide your next action in JSON format:
    {{
        "action": "action_name",
        "parameters": {{
            "param1": "value1",
            "param2": "value2"
        }}
    }}
    """
    return prompt

def generate_response(prompt):
    """Sends the prompt to the LLM and retrieves a response."""
    # Replace with your actual LLM call
    response = "Replace with LLM response"
    return response

def parse_response(response):
    """Extracts action and parameters from the LLM response."""
    try:
        response_json = json.loads(response)
        action = response_json["action"]
        parameters = response_json["parameters"]
    except (json.JSONDecodeError, KeyError):
        return None, None
    return action, parameters

def execute_action(action, parameters):
    """Executes the action with the given parameters."""
    if action == "list_files":
        files = os.listdir(parameters.get("directory", "."))
        return f"Files in directory: {', '.join(files)}"
    elif action == "read_file":
        try:
            with open(parameters["file_path"], "r") as f:
                content = f.read()
            return f"File content: {content}"
        except FileNotFoundError:
            return "File not found."
    # Add more actions as needed
    else:
        return "Invalid action."

def convert_result_to_string(result):
    """Formats the result into a string."""
    return str(result)

def agent_loop(memory, user_input, system_rules):
    """Main agent loop."""
    while True:
        prompt = construct_prompt(memory, user_input, system_rules)
        response = generate_response(prompt)
        action, parameters = parse_response(response)

        if action is None:
            print("Invalid response format.")
            break

        result = execute_action(action, parameters)
        result_string = convert_result_to_string(result)

        # Update memory, check for termination conditions, etc.
        memory += f"\nAction: {action}, Result: {result_string}"

        if action == "terminate":
            break

        # Get next user input or continue based on logic
        user_input = input("Enter your next query: ")

    print("Agent loop terminated.")

# Example usage
memory = ""
user_input = "List files in the current directory."
system_rules = "You can list files, read files, and answer questions about them."

agent_loop(memory, user_input, system_rules)